In [2]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
### YOUR CODE HERE

import tensorflow.keras.regularizers as reg

###
import tensorflow.keras.utils as ku 
import numpy as np 

In [3]:
tokenizer = Tokenizer()
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt \
    -O /tmp/sonnets.txt
data = open('/tmp/sonnets.txt').read()

corpus = data.lower().split("\n")


tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)


# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

--2020-02-15 20:51:40--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.124.128, 2607:f8b0:4001:c14::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.124.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘/tmp/sonnets.txt’

/tmp/sonnets.txt    100%[===================>]  91.38K  --.-KB/s    in 0.001s  

2020-02-15 20:51:40 (143 MB/s) - ‘/tmp/sonnets.txt’ saved [93578/93578]



In [9]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length = max_sequence_len - 1))
model.add(Bidirectional(LSTM(150, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words,
                kernel_regularizer=reg.l2(0.01),
                activity_regularizer=reg.l1(0.01),
                activation='relu'))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)

model.compile(loss='categorical_crossentropy', optimizer = adam, metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 10, 100)           321100    
_________________________________________________________________
bidirectional_6 (Bidirection (None, 10, 300)           301200    
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 300)           0         
_________________________________________________________________
bidirectional_7 (Bidirection (None, 300)               541200    
_________________________________________________________________
dense_4 (Dense)              (None, 3211)              966511    
_________________________________________________________________
dense_5 (Dense)              (None, 3211)              10313732  
Total params: 12,443,743
Trainable params: 12,443,743
Non-trainable params: 0
__________________________________________

In [0]:
 history = model.fit(predictors, label, epochs=100, verbose=1)

Train on 15462 samples
Epoch 1/100
15462/15462 [==============================] - 34s 2ms/sample - loss: 7.0558 - acc: 0.0206
Epoch 2/100
15462/15462 [==============================] - 32s 2ms/sample - loss: 6.5231 - acc: 0.0244
Epoch 3/100
15462/15462 [==============================] - 32s 2ms/sample - loss: 6.4425 - acc: 0.0302
Epoch 4/100
15462/15462 [==============================] - 33s 2ms/sample - loss: 6.3927 - acc: 0.0347
Epoch 5/100
15462/15462 [==============================] - 33s 2ms/sample - loss: 6.3499 - acc: 0.0356
Epoch 6/100
15462/15462 [==============================] - 33s 2ms/sample - loss: 6.3164 - acc: 0.0363
Epoch 7/100
15462/15462 [==============================] - 32s 2ms/sample - loss: 6.2910 - acc: 0.0389
Epoch 8/100
15462/15462 [==============================] - 33s 2ms/sample - loss: 6.2530 - acc: 0.0389
Epoch 9/100
15462/15462 [==============================] - 32s 2ms/sample - loss: 6.2115 - acc: 0.0419
Epoch 10/100
15462/15462 [========================

In [0]:
import matplotlib.pyplot as plt
acc = history.history['acc']
loss = history.history['loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.title('Training accuracy')

plt.figure()

plt.plot(epochs, loss, 'b', label='Training Loss')
plt.title('Training loss')
plt.legend()

plt.show()

In [0]:
seed_text = "Help me Obi Wan Kenobi, you're my only hope"
next_words = 100
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = model.predict_classes(token_list, verbose=0)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)